In [1]:
!pip install great_expectations==1.3.0 pandas opentelemetry-api==1.28.2 opentelemetry-distro==0.49b2 opentelemetry-exporter-otlp==1.28.2 opentelemetry-exporter-otlp-proto-grpc==1.28.2 opentelemetry-sdk==1.28.2 opentelemetry-semantic-conventions==0.49b2 opentelemetry-util-http==0.49b2 wrapt==1.16.0
import os, subprocess

In [2]:
DATA_PRODUCT_NAME = 'consuntiviDiProduzione'
QUALITY_SIDECAR_SCRIPT_BASE_PATH = 'quality_sidecar_script/'
EXPECTATIONS_JSON_FILE_PATH = 'resources/gx_v0.1.json'

In [4]:
os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'] = 'http://platform-collector:4317'
#os.environ['OTEL_METRIC_EXPORT_INTERVAL'] = '10000'
os.environ['DATA_PRODUCT_NAME'] = DATA_PRODUCT_NAME
os.environ['OTEL_SERVICE_NAME'] = f'{DATA_PRODUCT_NAME}-quality_sidecar'


command = f"opentelemetry-instrument --metrics_exporter otlp,console --logs_exporter otlp,console python {QUALITY_SIDECAR_SCRIPT_BASE_PATH}/app.py {QUALITY_SIDECAR_SCRIPT_BASE_PATH}/{EXPECTATIONS_JSON_FILE_PATH}"
subprocess.run(command, shell=True, check=True)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmpyuaro8up' for ephemeral docs site
INFO:root:Starting the application...
INFO:root:Reading GreatExpectations json file...
INFO:root:Setting up GreatExpectations...


INFO:root:Creating ValidationResults...
Calculating Metrics:  14%|█▎        | 3/22 [00:00<00:00, 158.24it/s]/opt/conda/lib/python3.11/site-packages/great_expectations/expectations/metrics/column_map_metrics/column_values_match_regex.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return column.astype(str).str.contains(regex)
Calculating Metrics: 100%|██████████| 22/22 [00:00<00:00, 420.05it/s]
INFO:root:ValidationResults created!
INFO:root:Creating ValidationResults...
Calculating Metrics:  14%|█▎        | 3/22 [00:00<00:00, 152.67it/s]/opt/conda/lib/python3.11/site-packages/great_expectations/expectations/metrics/column_map_metrics/column_values_match_regex.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return column.astype(str).str.contains(regex)
Calculating Metrics: 100%|██████████| 22/22 [00:00<

{
    "resource_metrics": [
        {
            "resource": {
                "attributes": {
                    "telemetry.sdk.language": "python",
                    "telemetry.sdk.name": "opentelemetry",
                    "telemetry.sdk.version": "1.28.2",
                    "service.name": "consuntiviDiProduzione-quality_sidecar",
                    "telemetry.auto.version": "0.49b2"
                },
                "schema_url": ""
            },
            "scope_metrics": [
                {
                    "scope": {
                        "name": "__main__",
                        "version": "",
                        "schema_url": "",
                        "attributes": null
                    },
                    "metrics": [
                        {
                            "name": "consuntividiproduzione-cdpdatasourcesample-cdpdataassetsample",
                            "description": "Validation results for suite: consuntiviDiProduzione-cdpDat

CompletedProcess(args='opentelemetry-instrument --metrics_exporter otlp,console --logs_exporter otlp,console python quality_sidecar_script//app.py quality_sidecar_script//resources/gx_v0.1.json', returncode=0)